# ✈️ Flight Delays Analysis – Exploratory Data Analysis (EDA)

This notebook is part of the **`flight-delays-analysis`** project. The goal is to analyze real-world flight delay data from the United States to uncover meaningful patterns and develop a basic predictive model.

---

## 🎯 Notebook Objectives

- Load the raw dataset
- Understand the structure and quality of the data
- Detect missing values, duplicates, and irrelevant features
- Identify initial patterns related to delays and cancellations

---

## 📁 Dataset Overview

The dataset contains flight records with variables such as:

- Airline, origin, destination
- Scheduled departure and arrival times
- Delay causes (weather, security, etc.)
- Cancellation information

Source: [Kaggle – Airline Delay Causes](https://www.kaggle.com/datasets/giovamata/airlinedelaycauses)

---

## 🧠 Notes

- This EDA focuses solely on **delay behavior**, not external economic or meteorological data.
- Key findings will be used later as a foundation for modeling and reporting.

---

📌 Author: **Josekawa** – 2025  
🔗 GitHub Repository: [github.com/Josekawa/flight-delays-analysis](https://github.com/Josekawa/flight-delays-analysis)



## 🧭 Project Workflow

To structure the analysis and keep things focused, I’m following a clear workflow that covers everything from data loading to exporting a cleaned dataset for modeling. Here's the step-by-step plan:

1. **Load the dataset**  
   Import the CSV file and confirm it loads correctly using Pandas.

2. **Initial inspection**  
   Get a general sense of the structure and contents of the dataset using `.info()`, `.describe()`, and other quick checks.

3. **Data cleaning**  
   Drop irrelevant or redundant columns, handle missing values, and create a proper `Date` column for time-based analysis.

4. **Feature documentation**  
   Record what each remaining column means, which ones are useful, and why I’ve kept them.

5. **Exploratory visuals**  
   Use charts to explore patterns in delays, cancellations, carriers, days of the week, and other relevant dimensions.

6. **Correlation analysis**  
   Compare different delay causes and examine their relationship with arrival delay using a correlation matrix.

7. **Save the cleaned dataset**  
   Export a cleaned version of the data to the `/data/processed/` folder for use in the modeling phase.



In [36]:
# 📦 Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setup for visualizations
sns.set_palette('pastel')
%matplotlib inline

# 📄 Load the dataset
file_path = '../data/raw/DelayedFlights.csv'
df = pd.read_csv(file_path)

# 🔍 Quick preview
print(f"Shape of dataset: {df.shape}")
df.head()



Shape of dataset: (1936758, 30)


,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [37]:
# Información general del dataset
df.info()

# Estadísticas de columnas numéricas
df.describe()

# Ver nombres de columnas
df.columns

# Ver tamaño del dataset
df.shape

# Comprobar nulos
df.isnull().sum()

# Ver duplicados
df.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 30 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   Year               int64  
 2   Month              int64  
 3   DayofMonth         int64  
 4   DayOfWeek          int64  
 5   DepTime            float64
 6   CRSDepTime         int64  
 7   ArrTime            float64
 8   CRSArrTime         int64  
 9   UniqueCarrier      object 
 10  FlightNum          int64  
 11  TailNum            object 
 12  ActualElapsedTime  float64
 13  CRSElapsedTime     float64
 14  AirTime            float64
 15  ArrDelay           float64
 16  DepDelay           float64
 17  Origin             object 
 18  Dest               object 
 19  Distance           int64  
 20  TaxiIn             float64
 21  TaxiOut            float64
 22  Cancelled          int64  
 23  CancellationCode   object 
 24  Diverted           int64  
 25  CarrierDelay      

0

The `ArrDelay` column ranges from -89 to over 1400 minutes. Most delays seem mild, but there are extreme outliers. This might affect visualizations or model performance later if not handled properly.


## ✍️ First Impressions from the Initial Inspection

- The dataset is large — almost 2 million rows and over 30 columns. That provides a rich set of information but requires careful focus to avoid unnecessary complexity.
- Time-related columns like `DepTime` and `ArrTime` are stored as floats. This suggests potential formatting issues or missing values that may need conversion or removal.
- Several delay-related fields (`CarrierDelay`, `WeatherDelay`, etc.) may contain NaNs when no delay occurred. This is logically consistent, but I’ll need to confirm that before deciding how to treat them.
- `Cancelled` and `Diverted` are binary columns indicating whether the flight occurred as planned. These will be important for filtering later to avoid bias in the delay analysis.
- Columns like `FlightNum`, `TailNum`, and `CRSElapsedTime` may not directly help with modeling but could offer insights during EDA.
- The column `Unnamed: 0` appears to be an autogenerated index — not useful, and likely safe to drop.

Overall, the dataset looks rich and fairly well structured. With the right cleanup and filtering, it should be well suited for exploratory and predictive analysis.



## 🔧 Initial Cleanup and Date Construction

In this section, we remove columns that are not analytically useful and create a proper datetime column by combining `Year`, `Month`, and `DayofMonth`. This will simplify temporal grouping and visualization.


In [38]:
# Check if date components exist
date_cols = ['Year', 'Month', 'DayofMonth']
if all(col in df.columns for col in date_cols):
    # Build the Date column
    df['Date'] = pd.to_datetime(
        df[date_cols].rename(columns={'Year': 'year', 'Month': 'month', 'DayofMonth': 'day'})
    )
    # Drop the original date parts
    df.drop(columns=date_cols, inplace=True)

# Move 'Date' to the first column if it exists
if 'Date' in df.columns:
    # Reorder columns
    cols = ['Date'] + [col for col in df.columns if col != 'Date']
    df = df[cols]

# Drop columns that do not add analytical value
columns_to_drop = [
    'Unnamed: 0', 'FlightNum', 'TailNum', 'TaxiIn', 'TaxiOut',
    'AirTime', 'DepTime', 'CRSDepTime', 'CRSArrTime',
    'ActualElapsedTime', 'CRSElapsedTime', 'Year', 'Month', 'DayofMonth', 'DayOfWeek'
]
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Preview the result
df.head()



,Date,ArrTime,UniqueCarrier,ArrDelay,DepDelay,Origin,Dest,Distance,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,2211.0,WN,-14.0,8.0,IAD,TPA,810,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008-01-03,1002.0,WN,2.0,19.0,IAD,TPA,810,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008-01-03,804.0,WN,14.0,8.0,IND,BWI,515,0,N,0,NaN,NaN,NaN,NaN,NaN
3,2008-01-03,1959.0,WN,34.0,34.0,IND,BWI,515,0,N,0,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,2121.0,WN,11.0,25.0,IND,JAX,688,0,N,0,NaN,NaN,NaN,NaN,NaN


## 📄 Column Definitions (for reference)

Before diving deeper, I want to document what each column represents — both for myself and anyone else reading this later. I've already dropped a number of columns that were redundant, constant, or not relevant for the type of analysis I want to do (like raw timestamps, flight IDs, and distance). 

Here’s a quick overview of the key variables I’ve kept in the dataset:

- `Date`: Combined from Year, Month, and DayofMonth
- `UniqueCarrier`: Airline code
- `Origin`: Origin airport (IATA)
- `Dest`: Destination airport (IATA)
- `ArrDelay`: Arrival delay in minutes  
- `CancellationCode`: Reason (A = carrier, B = weather, C = NAS, D = security)
- `Diverted`: Whether the flight was diverted from its scheduled destination (1 = yes, 0 = no)
- `CarrierDelay`: Airline responsibility (e.g. maintenance, crew)
- `WeatherDelay`: Due to hazardous weather
- `NASDelay`: NAS-related (air traffic, runways, etc.)
- `SecurityDelay`: Security-related issues
- `LateAircraftDelay`: Caused by late arrival of aircraft from a previous flight

This list will help me stay focused on the features that matter most for EDA, modeling, and visual storytelling.


## 🧼 Handling Missing Values and Duplicates

This section identifies and removes rows with missing values in critical columns, as well as duplicate records, to ensure the integrity and quality of the dataset before analysis.


In [39]:
delay_cols = [
    'CarrierDelay', 'WeatherDelay', 'NASDelay',
    'SecurityDelay', 'LateAircraftDelay'
]
df[delay_cols] = df[delay_cols].fillna(0)

# Drop rows with any missing values
df.dropna(inplace=True)

# Drop duplicate rows
print(f"Duplicate rows before cleaning: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Duplicate rows after cleaning: {df.duplicated().sum()}")

# Keep only flights that were completed and not diverted
df = df[(df['Cancelled'] == 0) & (df['Diverted'] == 0)]

# Optional: drop the columns if you no longer need them
df.drop(columns=['Cancelled', 'Diverted'], inplace=True)

# Confirm result
print(f"Remaining flights: {len(df)}")


# Final overview
df.head()


Duplicate rows before cleaning: 2
Duplicate rows after cleaning: 0
Remaining flights: 1928369


,Date,ArrTime,UniqueCarrier,ArrDelay,DepDelay,Origin,Dest,Distance,CancellationCode,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,2211.0,WN,-14.0,8.0,IAD,TPA,810,N,0.0,0.0,0.0,0.0,0.0
1,2008-01-03,1002.0,WN,2.0,19.0,IAD,TPA,810,N,0.0,0.0,0.0,0.0,0.0
2,2008-01-03,804.0,WN,14.0,8.0,IND,BWI,515,N,0.0,0.0,0.0,0.0,0.0
3,2008-01-03,1959.0,WN,34.0,34.0,IND,BWI,515,N,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,2121.0,WN,11.0,25.0,IND,JAX,688,N,0.0,0.0,0.0,0.0,0.0
